# Homework 5

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from collections import namedtuple, defaultdict
from typing import Optional, List
import heapq as hq
import numpy as np
import math
import random


def grid_successor_fn(state):
  """Helper for testing heuristic search.
  """
  arrival_costs = np.array([
    [1, 1, 8, 1, 1],
    [1, 8, 1, 1, 1],
    [1, 8, 1, 1, 1],
    [1, 1, 1, 8, 1],
    [1, 1, 2, 1, 1],
  ], dtype=float)

  act_to_delta = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1),
  }

  r, c = state

  for act, (dr, dc) in act_to_delta.items():
    new_r, new_c = r + dr, c + dc
    # Check if in bounds
    if not (0 <= new_r < arrival_costs.shape[0] and \
            0 <= new_c < arrival_costs.shape[1]):
      continue
    # Valid action
    yield (act, (new_r, new_c), arrival_costs[new_r, new_c])


def grid_check_goal_fn(state):
  """Helper for testing heuristic search.
  """
  # Bottom right corner of grid
  return state == (4, 4)


def grid_heuristic_fn(state):
  """Helper for testing heuristic search.
  """
  # Manhattan distance
  return abs(state[0] - 4) + abs(state[1] - 4)


# A useful data structure for heuristic search
Node = namedtuple("Node", ["state", "parent", "action", "cost", "g"])

# Useful geometric data structures and methods.
def euclidean_distance(a, b):
  return math.sqrt((a.x - b.x) ** 2 + (a.y - b.y) ** 2)


RobotConfig = namedtuple('RobotConfig', ['x', 'y'])


class Circle(namedtuple('Circle', ['x', 'y', 'r'])):
  def collide(self, config: RobotConfig):
    return euclidean_distance(config, self) < self.r


class RobotPlanningProblem:
  def __init__(self):
    self.x_range = (-10, 10)  # range of the x value.
    self.y_range = (-10, 10)  # range of the y value.
    self.obstacles = [
      Circle(0, 0, 4),
      Circle(-5, -5, 2),
      Circle(3, -4, 3),
      Circle(-3, 4, 1)
    ]

  def sample(self):
    """Generate a random sample in the robot configuration space."""
    return RobotConfig(random.uniform(*self.x_range), random.uniform(*self.y_range))

  def distance(self, a: RobotConfig, b: RobotConfig):
    """Return the distance between two robot configurations."""
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

  def collide(self, x: RobotConfig):
    """Test if the input robot configuration collides with some obstacles."""
    for obs in self.obstacles:
      if obs.collide(x):
        return True
    return False


class RRTNode:
  """Node of a rapidly-exploring random tree."""

  config: RobotConfig # Robot state
  parent: Optional['RRTNode']  # parent node
  children: List['RRTNode']  # children

  def __init__(self, config: RobotConfig, parent: Optional['RRTNode'] = None):
    self.config = config
    self.parent = parent
    self.children = list()

  def add_child(self, other):
    """Register another node as the child of the current node."""
    self.children.append(other)
    return self

  def attach_to(self, other):
    """Attach the current node to another node."""
    self.parent = other
    self.parent.add_child(self)
    return self

  def backtrace(self, config=True):
    """Return the path from the current node to the root.

    Args:
      config (bool): if True, return only the robot configurations.
      Otherwise, return the RRTNode list.
    """
    path = list()

    def dfs(x):
      nonlocal path
      if x.parent is not None:
        dfs(x.parent)
      path.append(x.config if config else x)

    try:
      dfs(self)
      return path
    finally:
      del dfs

  def __repr__(self):
      return f'RRTNode(config={self.config}, parent={self.parent})'


class RRT:
  """The RRT tree."""

  def __init__(self, problem: RobotPlanningProblem, roots: List[RRTNode]):
    if isinstance(roots, RRTNode):
      roots = [roots]
    else:
      assert isinstance(roots, list)

    self.problem = problem
    self.roots = roots
    self.size = len(roots)

  def extend(self, parent: RRTNode, child_config: RobotConfig):
    """Create a new RRTNode from a robot configuration and set its parent."""
    child = RRTNode(child_config).attach_to(parent)
    self.size += 1
    return child

  def nearest(self, config: RobotConfig):
    """Return the RRTNode in the tree that is closest to the target configuration."""
    best_node, best_value = None, np.inf
    for node in traverse_rrt_bfs(self.roots):
      distance = self.problem.distance(node.config, config)
      if distance < best_value:
        best_value = distance
        best_node = node

    return best_node


def traverse_rrt_bfs(nodes):
  """Run breath-first search to return all RRTNodes that are descendants of
  the input list of RRTNodes.

  Args:
    nodes: a list of RRTNodes as the seed nodes.

  Returns:
    a list of RRTNodes that are descendants of the input lists (including the input list).
  """
  queue = nodes.copy()
  results = list()

  while len(queue) > 0:
    x = queue[0]
    queue = queue[1:]
    results.append(x)
    for y in x.children:
      queue.append(y)

  return results


def visualize_map(problem, canvas_size=(512, 512)):
  """Visualize a 2D environment.

  Args:
    problem (RobotPlanningProblem): the robot planning problem.
    canvas_size (Tuple[int]): width and height of the canvas.

  Returns:
    a PIL image of the visualization.
  """
  from PIL import Image, ImageDraw

  image = Image.new('RGB', canvas_size, color='white')
  draw = ImageDraw.Draw(image)

  def gen_xy(x, y):
    return (
      int((x - problem.x_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[0]),
      canvas_size[1] - int((y - problem.y_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[1])
    )

  for obs in problem.obstacles:
    if isinstance(obs, Circle):
      AABB = (*gen_xy(obs.x - obs.r, obs.y + obs.r), *gen_xy(obs.x + obs.r, obs.y - obs.r))
      draw.ellipse(AABB, fill='gray', outline='gray')

  return image


def visualize_path(problem: RobotPlanningProblem, path: List[RobotConfig], canvas_size=(512, 512)):
  """Visualize a path in a 2D environment.

  Args:
    problem (RobotPlanningProblem): the robot planning problem.
    path (List[RobotConfig]): a list of RobotConfig.
    canvas_size (Tuple[int]): width and height of the canvas.

  Returns:
    a PIL image of the visualization.
  """
  from PIL import Image, ImageDraw

  image = visualize_map(problem, canvas_size=canvas_size)
  draw = ImageDraw.Draw(image)

  def gen_xy(x, y):
    return (
      int((x - problem.x_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[0]),
      canvas_size[1] - int((y - problem.y_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[1])
    )

  for config1, config2 in zip(path[:-1], path[1:]):
    draw.line((*gen_xy(config1.x, config1.y), *gen_xy(config2.x, config2.y)), fill='red')
  return image


def visualize_rrts(problem: RobotPlanningProblem, rrts: List[RRT], canvas_size=(512, 512)):
  """Visualize RRTs in a 2D environment.

  Args:
    problem (RobotPlanningProblem): the robot planning problem.
    rrts (List[RRT]): a list of RRTs. Currently the function supports up to two rrts. The first one will show in red,
      while the second one wil show in blue.
    canvas_size (Tuple[int]): width and height of the canvas.

  Returns:
    a PIL image of the visualization.
  """
  from PIL import Image, ImageDraw

  image = visualize_map(problem, canvas_size=canvas_size)
  draw = ImageDraw.Draw(image)

  def gen_xy(x, y):
    return (
      int((x - problem.x_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[0]),
      canvas_size[1] - int((y - problem.y_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[1])
    )

  for rrt, color in zip(rrts, ['red', 'blue']):
    for node in traverse_rrt_bfs(rrt.roots):
      if node.parent is not None:
        draw.line((*gen_xy(node.parent.config.x, node.parent.config.y), *gen_xy(node.config.x, node.config.y)), fill=color)
  return image


def check_path_smooth(problem, path, max_diff, eps=1e-5):
  """Check if a path is collision-free and satisfies the max_diff constraint."""
  for config1, config2 in zip(path[:-1], path[1:]):
    if any(abs(config1[i] - config2[i]) > max_diff + eps for i in range(len(config1))):
      return False
  for config in path:
    if problem.collide(config):
      return False
  return True


def check_path_smooth_and_successful(problem, path, initial_config, goal_config, max_diff, eps=1e-5):
  """Check if a path is collision-free, satisfies the max_diff constraint, and connects initial_config and goal_config."""
  return np.allclose(path[0], initial_config, atol=eps) and np.allclose(path[-1], goal_config, atol=eps) and check_path_smooth(problem, path, max_diff, eps=eps)





## Problems

### Heuristic Search
Complete an implementation of heuristic search, encompassing A*, GBFS, or UCS.

For reference, our solution is **45** lines of code.

In [ ]:
def run_heuristic_search(initial_state, check_goal, get_successors, get_priority, max_expansions=1000):
  """A generic heuristic implementation.

  Depending on get_priority, can implement A*, GBFS, or UCS.

  Important: for determinism (and to make sure our tests pass),
  please break ties using the state itself. For example,
  if you would've otherwise sorted by get_priority(node), you
  should now sort by (get_priority(node), node.state).

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    get_priority: A callable that takes a Node and returns a
        float priority, with lower better, for the priority
        queue. This function is what switches between different
        versions of heurstic search.
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# We will test this implementation more thoroughly with the
# specific heuristic search algorithms that follow
initial_state = (0, 0)
get_priority_fn = lambda node: 0
result = run_heuristic_search(initial_state, grid_check_goal_fn,
    grid_successor_fn, get_priority_fn)
assert len(result) == 4

print('Tests passed.')

### Uniform Cost Search
Use your implementation of `run_heuristic_search` to implement uniform cost search.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_uniform_cost_search(initial_state, check_goal, get_successors, max_expansions=1000):
  """Uniform-cost search.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# If your results do not match the expected ones, make sure that you are tiebreaking
# as described in the docstring for `run_heuristic_search`.
initial_state = (0, 0)
state_sequence, action_sequence, cost_sequence, num_expansions = run_uniform_cost_search(
    initial_state, grid_check_goal_fn, grid_successor_fn)
assert state_sequence == [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
assert action_sequence == ['down', 'down', 'down', 'right', 'right', 'down', 'right', 'right']
assert cost_sequence == [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]
assert num_expansions == 22

print('Tests passed.')

### A* Search
Use your implementation of `run_heuristic_search` to implement A* search.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_astar_search(initial_state, check_goal, get_successors, heuristic, max_expansions=1000):
  """A* search.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    heuristic: A callable that takes a state and returns an
        estimated cost-to-go (must be nonnegative).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# If your results do not match the expected ones, make sure that you are tiebreaking
# as described in the docstring for `run_heuristic_search`.
initial_state = (0, 0)
state_sequence, action_sequence, cost_sequence, num_expansions = run_astar_search(
    initial_state, grid_check_goal_fn, grid_successor_fn, grid_heuristic_fn)
assert state_sequence == [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
assert action_sequence == ['down', 'down', 'down', 'right', 'right', 'down', 'right', 'right']
assert cost_sequence == [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]
assert num_expansions == 11

print('Tests passed.')

### Greedy Best-First Search
Use your implementation of `run_heuristic_search` to implement GBFS.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_greedy_best_first_search(initial_state, check_goal, get_successors, heuristic, max_expansions=1000):
  """GBFS.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    heuristic: A callable that takes a state and returns an
        estimated cost-to-go (must be nonnegative).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# If your results do not match the expected ones, make sure that you are tiebreaking
# as described in the docstring for `run_heuristic_search`.
initial_state = (0, 0)
state_sequence, action_sequence, cost_sequence, num_expansions = run_greedy_best_first_search(
    initial_state, grid_check_goal_fn, grid_successor_fn, grid_heuristic_fn)
assert state_sequence == [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
assert action_sequence == ['right', 'right', 'right', 'right', 'down', 'down', 'down', 'down']
assert cost_sequence == [1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
assert num_expansions == 8

print('Tests passed.')

### Extend Path in a Robot Configuration Space
Extend path in a robot configuration space.
In this section, you will implement an important utility for doing motion planning in a continuous space: extend_path.
The function will take two arguments, `initial_config` and `goal_config` and make a linear interpolation between this two configurations in the
robot's configuration space. Its output is a path, that is, a list of robot configurations along the line from `initial_config` to `goal_config`.
When the `initial_config` and `goal_config` cannot be directly connected with a line, your function should return a ``prefix'' of the path.
In other words, you should try to move towards the goal until you reach some obstacles in the environment.


For reference, our solution is **15** lines of code.

In [ ]:
def extend_path(problem, initial_config, goal_config, max_diff):
  """Construct a linear path in the configuration space from the initial configuration to the goal configuration.
  This can be achieved by doing a linear interpolation between initial_config and goal_config.
  At each step, the movement along x and along y should be both smaller than or equal to max_diff. Although we will
  only test your algorithm on the 2-dim robot configuration spaces, we strongly encourage you to implement this
  function for any dimensional robot configuration spaces.

  While extending from the initial configuration to the goal configuration, if a configuration triggers a collision
  with an obstacle, your algorithm should return the longest prefix that does not involve any collision. See the 
  starter code for details.

  Args:
    problem (RobotPlanningProblem): an RobotPlanningProblem instance, used for collision checking.
    initial_config (RobotConfig): the initial configuration. It should be the first element of your return list.
    goal_config (RobotConfig): the target configuration. It should be the last element of your return list.
    max_diff (float): the maximum movement in x or y.

  Returns:
    successful: a bool value indicating whether we have found a path from the initial configuration to the goal configuration.
    path: a list of RobotConfig: extending from the initial to the goal configuration.
  """
  path = [initial_config]
  # Step 1: Generate a list of robot configurations that connects initial_config and goal_config.
  # Your code only need to take care of max_diff. You can ignore collision checking for now.
  # Your code here...
  assert path[-1] == goal_config

  # Step 2: Collision checking.
  for i, config in enumerate(path):
    if problem.collide(config):
      return False, path[:i]
  return True, path

Tests

In [ ]:
def extend_path_test1():
  return 1
  problem = RobotPlanningProblem()
  init_config = RobotConfig(-8, -8)
  goal_config = RobotConfig(8, 8)

  success, path = extend_path(problem, init_config, goal_config, 0.1)
  assert not success     # you can't reach the goal with a single linear motion.
  assert len(path) > 10  # you should make some progress.
  assert check_path_smooth(problem, path, 0.1)

extend_path_test1()
def extend_path_test2():
  return 1
  problem = RobotPlanningProblem()
  init_config = RobotConfig(-8, -8)
  goal_config = RobotConfig(-7, 6)

  success, path = extend_path(problem, init_config, goal_config, 0.1)
  assert success
  assert check_path_smooth_and_successful(problem, path, init_config, goal_config, 0.1)

extend_path_test2()
print('Tests passed.')

### Extend Path Visualization

We have created a visualization tool for you in colab to visualize the path. You can run the following code blocks
and they will generate the visualized the map as well as the path.


In [ ]:
problem = RobotPlanningProblem()
init_config = RobotConfig(-8, -8)
goal_config = RobotConfig(8, 8)

success, path = extend_path(problem, init_config, goal_config, 0.1)
visualize_path(problem, path)

### RRT
In this section, you will implement the RRT algorithm we talked about in the class to find a path from one configuration to the goal configuration
in a 2D environment.

**Hint**: there is a hyperparameter in RRT, which balances how frequently we want to test if the goal configuration can be reached from a node in the current
RRT tree. You probably need to tune this hyperparameter a bit to make sure the algorithm terminates within a decent number of iterations. Our suggested value
for this value is 0.1.


For reference, our solution is **28** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `extend_path`. You may not need to use all of them.

In [ ]:
def rrt(problem, initial_config, goal_config, max_diff, nr_iterations=100):
  """Implement RRT algorithm to compute a path from the initial configuration to the target configuration.
  Your implementation should returns a list of robot configurations, starting from initial_config and terminating at goal_config.

  Similar to the definition in extend_path, at each step, the movement along x and along y should be both smaller than or equal to max_diff.
  Thus, you can use the extend_path utility that you just implemented to check whether there is a path from one node to another.
  After generating the result, you can visualize your RRT's by calling the visualize_rrts function provided.

  Args:
    problem (RobotPlanningProblem): an RobotPlanningProblem instance, used for configuration sampling and collision checking.
    initial_config (RobotConfig): the initial configuration. It should be the first element of your return list.
    goal_config (RobotConfig): the target configuration. It should be the last element of your return list.
    max_diff (float): the maximum movement in x or y.
    nr_iterations (int): the number of iterations in RRT.

  Returns:
    path: a list of RobotConfig as the trajectory. Return None if we can't find such a path after the specified number of iterations.
    rrt: an RRT object. The root should be the intial_config. There must also exist another node for goal_config.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def rrt_test():
  problem = RobotPlanningProblem()
  init_config = RobotConfig(-8, -8)
  goal_config = RobotConfig(8, 8)

  path, tree = rrt(problem, init_config, goal_config, 0.1)
  assert check_path_smooth_and_successful(problem, path, init_config, goal_config, 0.1)

rrt_test()
print('Tests passed.')

### RRT Visualization

We have created a visualization tool for you in colab to visualize the RRT tree during your search. You can run the following code blocks
and they will generate the visualized the map as well as some lines showing the edges in the RRT.

If you run this cell multiple times, you will see different RRT trees due to different random seeds.
You will find that sometimes the tree has very few nodes -- indicating that your RRT algorithm finds a solution very quickly.
Sometimes it takes many iterations to find the solution. This suggests that the algorithm is very sensitive to random seeds,
which is bad for practical usage. Thus, in practice, we shuold always use improved RRT algorithm variants such as BiRRT (bidirectional RRT).


In [ ]:
problem = RobotPlanningProblem()
init_config = RobotConfig(-8, -8)
goal_config = RobotConfig(8, 8)

path, tree = rrt(problem, init_config, goal_config, 0.1)
visualize_rrts(problem, [tree])